
# Backtest: EMA(20/50/200) + ADX(14) Signals from Screener CSV

This notebook takes the **screener output CSV** (from the companion screener notebook) and runs a simple, rules-based backtest:

**Core assumptions**  
- **Capital per trade:** ₹50,000 (fixed).  
- **Entry:** On the **next trading day's open** after a signal date in the CSV.  
- **Exit:** When an **opposite signal** triggers for the same ticker (recomputed inside this notebook from price data), **or** a **time-based stop** (max holding days) — whichever comes first.  
- **No pyramiding / one position per ticker at a time.**  
- **Parallel trades allowed** across different tickers; each trade uses its own 50k allocation.

**Inputs expected from CSV**  
The screener CSV should contain at least these columns: `Ticker, Date, Signal` where `Signal` is `"LONG"` or `"SHORT"` and `Date` is the signal day in `YYYY-MM-DD`.

**Outputs**  
- `outputs/trade_log.csv`: each executed trade with entry/exit details and P&L  
- `outputs/equity_curve.csv`: daily aggregate equity assuming parallel trades each allocated ₹50k  
- `outputs/summary_metrics.csv`: consolidated performance metrics

> Notes
> - Data fetched via **yfinance** and timestamps normalized to **IST**.
> - Indicators computed with **pandas_ta** for exit detection.
> - Slippage/fees are **not** included; you can add them via parameters below.



## 0) Dependencies
If needed, install once (uncomment and run):
```python
# %pip install yfinance pandas pandas_ta numpy pytz
```


In [5]:

import os, math, warnings
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from zoneinfo import ZoneInfo

warnings.filterwarnings("ignore")
pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 200)

# =====================
# USER PARAMETERS
# =====================
INPUT_CSV = "outputs/swing_ema_adx_candidates_2024-12-31.csv"  # <-- set to your screener CSV path
OUT_DIR   = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

# Strategy / indicator params (should match screener)
EMA_FAST = 20
EMA_SLOW = 50
EMA_LONG = 200
ADX_LEN  = 14
ADX_THRESHOLD = 25.0

# Backtest behavior
CAPITAL_PER_TRADE = 50_000.0     # ₹50k per trade (fixed)
MAX_HOLD_BARS     = 15           # 2-3 weeks on daily bars ≈ 10-15 trading days
ALLOW_SHORTS      = True         # set False if you only want long trades
ENTRY_PRICE_COL   = "Open"       # enter at next day's open
EXIT_ON_OPPOSITE  = True         # exit when opposite signal appears
SLIPPAGE_PCT      = 0.0          # optional slippage (e.g., 0.0005 for 5 bps)
FEE_PER_TRADE     = 0.0          # flat fee per entry/exit (set as needed)

IST = ZoneInfo("Asia/Kolkata")



## 1) Load signals CSV
The CSV should have **Ticker, Date, Signal**. Dates are expected in `YYYY-MM-DD` (IST).


In [6]:

sig_df = pd.read_csv(INPUT_CSV)
required_cols = {"Ticker","Date","Signal"}
missing = required_cols - set(sig_df.columns)
if missing:
    raise ValueError(f"CSV missing required columns: {missing}")

# Normalize
sig_df["Date"] = pd.to_datetime(sig_df["Date"]).dt.tz_localize(IST, nonexistent='shift_forward', ambiguous='NaT')
sig_df["Signal"] = sig_df["Signal"].str.upper().str.strip()
sig_df = sig_df[sig_df["Signal"].isin(["LONG","SHORT"])].copy()
if not ALLOW_SHORTS:
    sig_df = sig_df[sig_df["Signal"] == "LONG"].copy()

tickers = sorted(sig_df["Ticker"].unique().tolist())
print(f"Signals loaded: {len(sig_df)} across {len(tickers)} tickers")
sig_df.head()


Signals loaded: 2 across 2 tickers


,Ticker,Date,Signal,Close,EMA20,EMA50,EMA200,ADX
0,LTIM.NS,2024-12-30 00:00:00+05:30,SHORT,5643.50,6087.63,6112.65,5771.53,32.00
1,TTML.NS,2024-12-30 00:00:00+05:30,SHORT,74.49,78.30,78.44,82.06,26.96



## 2) Fetch price data (IST) and compute indicators for exit rules
We re-compute EMA20/50/200 and ADX(14) to detect *opposite* signals for exits.


In [7]:

def fetch_ohlcv(ticker: str, start_date: pd.Timestamp) -> pd.DataFrame:
    # fetch from a few months before earliest signal for this ticker
    start = (start_date - pd.Timedelta(days=365*2)).date().isoformat()
    df = yf.download(ticker, start=start, interval="1d", auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty:
        return df
    if df.index.tz is None:
        df.index = df.index.tz_localize("UTC").tz_convert(IST)
    else:
        df.index = df.index.tz_convert(IST)
    df = df.rename(columns=str.title)
    return df

def add_indicators(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    out = df.copy()
    out[f"EMA{EMA_FAST}"] = ta.ema(out["Close"], length=EMA_FAST)
    out[f"EMA{EMA_SLOW}"] = ta.ema(out["Close"], length=EMA_SLOW)
    out[f"EMA{EMA_LONG}"] = ta.ema(out["Close"], length=EMA_LONG)
    adx = ta.adx(out["High"], out["Low"], out["Close"], length=ADX_LEN)
    if adx is not None and not adx.empty:
        out["ADX"] = adx[f"ADX_{ADX_LEN}"]
    else:
        out["ADX"] = np.nan
    # Crossover flags for exit detection
    ef, es = f"EMA{EMA_FAST}", f"EMA{EMA_SLOW}"
    out["bull_cross"] = (out[ef] > out[es]) & (out[ef].shift(1) <= out[es].shift(1))
    out["bear_cross"] = (out[ef] < out[es]) & (out[ef].shift(1) >= out[es].shift(1))
    return out



## 3) Backtest engine
- Entry at next bar's open after a signal date
- Exit on opposite crossover (**with ADX filter**) or after `MAX_HOLD_BARS`
- Size = floor(₹50,000 / entry_price)


In [8]:

all_trades = []

for tk in tickers:
    # Signals for this ticker
    s = sig_df[sig_df["Ticker"] == tk].sort_values("Date")
    if s.empty: 
        continue

    # Fetch price series starting before earliest signal
    earliest = s["Date"].min()
    px = fetch_ohlcv(tk, earliest)
    if px.empty or len(px) < EMA_LONG + 5:
        print(f"[WARN] No data or too short for {tk}")
        continue

    px = add_indicators(px).dropna(subset=[f"EMA{EMA_LONG}","ADX"])

    # Helper to find opposite signal index after an entry index
    ef, es, el = f"EMA{EMA_FAST}", f"EMA{EMA_SLOW}", f"EMA{EMA_LONG}"
    def opposite_trig(row_signal, df_slice):
        if row_signal == "LONG":
            # opposite is bear_cross with bearish context and ADX filter
            mask = (df_slice[ef] < df_slice[es]) & df_slice["bear_cross"] & (df_slice["Close"] < df_slice[el]) & (df_slice["ADX"] >= ADX_THRESHOLD)
        else:
            # opposite is bull_cross with bullish context and ADX filter
            mask = (df_slice[ef] > df_slice[es]) & df_slice["bull_cross"] & (df_slice["Close"] > df_slice[el]) & (df_slice["ADX"] >= ADX_THRESHOLD)
        hits = df_slice[mask]
        return hits.index[0] if not hits.empty else None

    # Prevent overlapping positions for same ticker
    in_pos = False
    last_exit_dt = None

    for _, row in s.iterrows():
        sig_dt = row["Date"]

        # Find the bar right AFTER signal date
        try:
            entry_bar_loc = px.index.get_indexer([sig_dt], method="nearest")[0]
            # ensure we choose strictly after the signal date (next bar)
            if px.index[entry_bar_loc] <= sig_dt:
                entry_bar_loc += 1
        except Exception:
            continue

        if entry_bar_loc >= len(px):
            continue

        entry_dt = px.index[entry_bar_loc]
        if last_exit_dt is not None and entry_dt <= last_exit_dt:
            # Skip signals that occur before/at the previous exit (avoid overlap from multiple signals close by)
            continue

        direction = row["Signal"]
        if (direction == "SHORT") and (not ALLOW_SHORTS):
            continue

        # Entry conditions re-check (ensure context still valid at entry bar)
        bar = px.iloc[entry_bar_loc]
        if direction == "LONG":
            ok = (bar["Close"] > bar[el]) and (bar[ef] > bar[es]) and (bar["ADX"] >= ADX_THRESHOLD)
        else:
            ok = (bar["Close"] < bar[el]) and (bar[ef] < bar[es]) and (bar["ADX"] >= ADX_THRESHOLD)

        if not ok:
            continue

        # Compute entry price including slippage
        entry_price = float(bar[ENTRY_PRICE_COL]) * (1 + SLIPPAGE_PCT if direction=="LONG" else 1 - SLIPPAGE_PCT)
        qty = math.floor(CAPITAL_PER_TRADE / entry_price)
        if qty <= 0:
            continue

        entry_idx = entry_bar_loc
        exit_idx = None

        # Time-based exit cutoff
        max_exit_idx = min(len(px)-1, entry_idx + MAX_HOLD_BARS)

        # Search for opposite signal after entry (if enabled)
        if EXIT_ON_OPPOSITE:
            opp_dt = opposite_trig(direction, px.iloc[entry_idx+1:max_exit_idx+1])
            if opp_dt is not None:
                # snap to next bar's open after opposite signal
                oi = px.index.get_loc(opp_dt)
                exit_idx = min(oi + 1, len(px)-1)

        if exit_idx is None:
            # Fallback to max hold exit at next bar
            exit_idx = max_exit_idx

        exit_bar = px.iloc[exit_idx]
        exit_price = float(exit_bar["Open"]) * (1 - SLIPPAGE_PCT if direction=="LONG" else 1 + SLIPPAGE_PCT)

        # P&L (no fees) then subtract fees
        if direction == "LONG":
            gross = (exit_price - entry_price) * qty
        else:
            gross = (entry_price - exit_price) * qty

        net = gross - (2 * FEE_PER_TRADE)  # entry + exit fees

        all_trades.append({
            "Ticker": tk,
            "Direction": direction,
            "EntryDate": entry_dt.isoformat(),
            "EntryPrice": round(entry_price, 4),
            "Qty": int(qty),
            "ExitDate": px.index[exit_idx].isoformat(),
            "ExitPrice": round(exit_price, 4),
            "GrossPnL": round(gross, 2),
            "NetPnL": round(net, 2),
            "ReturnPct_on_50k": round(net / CAPITAL_PER_TRADE * 100.0, 3),
            "BarsHeld": int(exit_idx - entry_idx),
        })

        last_exit_dt = px.index[exit_idx]



## 4) Results & Metrics
We compile the trade log, compute a simple equity curve (parallel trades, each funded with ₹50k), and summary stats.


In [9]:

if not all_trades:
    print("No trades generated. Check CSV path, signals, or parameters.")
    trade_log = pd.DataFrame(columns=["Ticker","Direction","EntryDate","EntryPrice","Qty","ExitDate","ExitPrice","GrossPnL","NetPnL","ReturnPct_on_50k","BarsHeld"])
else:
    trade_log = pd.DataFrame(all_trades)

# Save trade log
trade_log_path = os.path.join(OUT_DIR, "trade_log.csv")
trade_log.to_csv(trade_log_path, index=False)
print(f"Saved trade log -> {trade_log_path} with {len(trade_log)} trades")
trade_log.head()


Saved trade log -> outputs/trade_log.csv with 2 trades


,Ticker,Direction,EntryDate,EntryPrice,Qty,ExitDate,ExitPrice,GrossPnL,NetPnL,ReturnPct_on_50k,BarsHeld
0,LTIM.NS,SHORT,2024-12-30T05:30:00+05:30,5680.4502,8,2025-01-20T05:30:00+05:30,5885.0,-1636.4,-1636.4,-3.273,15
1,TTML.NS,SHORT,2024-12-30T05:30:00+05:30,76.5000,653,2025-01-20T05:30:00+05:30,77.0,-326.5,-326.5,-0.653,15


In [10]:

# Build a naive equity curve assuming each trade is independent and funded separately with 50k.
# We accumulate PnL on the exit date.
if trade_log.empty:
    equity_curve = pd.DataFrame(columns=["Date","Equity"])
else:
    # Aggregate net PnL by exit date
    tl = trade_log.copy()
    tl["ExitDate"] = pd.to_datetime(tl["ExitDate"])
    pnl_by_day = tl.groupby(tl["ExitDate"].dt.date)["NetPnL"].sum().sort_index()

    # Equity = cumulative PnL over time, starting at 0 (only tracking PnL, not total NAV)
    eq = pnl_by_day.cumsum()
    equity_curve = pd.DataFrame({"Date": pd.to_datetime(eq.index), "Equity": eq.values})

equity_path = os.path.join(OUT_DIR, "equity_curve.csv")
equity_curve.to_csv(equity_path, index=False)
print(f"Saved equity curve -> {equity_path}")
equity_curve.tail()


Saved equity curve -> outputs/equity_curve.csv


,Date,Equity
0,2025-01-20,-1962.9


In [11]:

# Summary metrics
def max_drawdown(series: pd.Series) -> float:
    if series.empty:
        return 0.0
    roll_max = series.cummax()
    dd = series - roll_max
    return float(dd.min())  # negative value

def summary(trades: pd.DataFrame, equity: pd.DataFrame) -> pd.DataFrame:
    if trades.empty:
        return pd.DataFrame([{
            "Trades": 0, "Wins": 0, "WinRate%": 0.0, "AvgNetPnL": 0.0, "MedianNetPnL": 0.0,
            "AvgRtn_on_50k_%": 0.0, "TotalNetPnL": 0.0, "ProfitFactor": np.nan,
            "MaxDrawdown": 0.0
        }])
    wins = (trades["NetPnL"] > 0).sum()
    losses = (trades["NetPnL"] < 0).sum()
    winrate = wins / len(trades) * 100.0
    total_net = trades["NetPnL"].sum()
    avg_net = trades["NetPnL"].mean()
    med_net = trades["NetPnL"].median()
    avg_rtn = trades["ReturnPct_on_50k"].mean()

    gross_profit = trades.loc[trades["NetPnL"] > 0, "NetPnL"].sum()
    gross_loss   = -trades.loc[trades["NetPnL"] < 0, "NetPnL"].sum()
    pf = (gross_profit / gross_loss) if gross_loss > 0 else np.nan

    mdd = max_drawdown(equity["Equity"]) if not equity.empty else 0.0

    return pd.DataFrame([{
        "Trades": int(len(trades)),
        "Wins": int(wins),
        "WinRate%": round(winrate, 2),
        "AvgNetPnL": round(avg_net, 2),
        "MedianNetPnL": round(med_net, 2),
        "AvgRtn_on_50k_%": round(avg_rtn, 3),
        "TotalNetPnL": round(total_net, 2),
        "ProfitFactor": round(pf, 3) if not np.isnan(pf) else np.nan,
        "MaxDrawdown": round(mdd, 2)
    }])

summary_df = summary(trade_log, equity_curve)
sum_path = os.path.join(OUT_DIR, "summary_metrics.csv")
summary_df.to_csv(sum_path, index=False)
print(f"Saved summary -> {sum_path}")
summary_df


Saved summary -> outputs/summary_metrics.csv


,Trades,Wins,WinRate%,AvgNetPnL,MedianNetPnL,AvgRtn_on_50k_%,TotalNetPnL,ProfitFactor,MaxDrawdown
0,2,0,0.0,-981.45,-981.45,-1.963,-1962.9,0.0,0.0
